In [1]:
using JuMP, Gurobi, GLMakie, Polyhedra, CDDLib

# Paper Reproduction: Two-Stage Robust Optimization

This notebook presents a Julia implementation of the column-and-constraint generation algorithm for solving two-stage robust optimization problems. The case study is a robust location-transportation problem, and the model settings can be found in the following original paper: 

**Reference**: *Zeng, Bo, and Long Zhao. "Solving two-stage robust optimization problems using a column-and-constraint generation method." Operations Research Letters 41.5 (2013): 457-461.*

---

#### Sets and Indices
- $\mathcal{I} = \{1,\dots,m\} $: facility sites (potential plants/warehouses)  
- $\mathcal{J} = \{1,\dots,n\} $: customers  
- Indices: $i \in \mathcal{I}$, $j \in \mathcal{J}$

#### Data (Parameters)
- $f_i \ge 0$: fixed opening cost at site $i$  
- $a_i \ge 0$: unit capacity installation cost at site $i$  
- $K_i \ge 0$: maximum installable capacity at site $i$  
- $c_{ij} \ge 0$: per-unit transportation cost from site $i$ to customer $j$
- $\bar d_j \ge 0$: nominal (baseline) demand of customer $j$  
- $\tilde d_j \ge 0$: maximum positive deviation of demand $j$ 
- $\Gamma \in \mathbb{Z}_+$: uncertainty budget controlling conservatism

In [2]:
m = 3
n = 3
f = [400, 414, 326]
a = [18, 25, 20]
K = 800
c = [22 33 24;
     33 23 30;
     20 25 27]
d_nom = [206.0, 274.0, 220.0]
d_dev = [40.0, 40.0, 40.0]
Γ = [1.8, 1.2]

data = Dict(:m=>m, :n=>n, :f=>f, :a=>a, :K=>K,
            :c=>c, :d_nom=>d_nom, :d_dev=>d_dev, :Γ=>Γ)

d_scenarios = Vector{Vector{Float64}}()
push!(d_scenarios, d_nom)

1-element Vector{Vector{Float64}}:
 [206.0, 274.0, 220.0]



#### Decision Variables (Complete List)
**First stage**
- $y_i \in \{0,1\}$ — open facility $i$ (1) or not (0)  
- $z_i \ge 0$ — installed capacity at facility $i$

**Second stage (recourse, depend on realized $d$)**
- $x_{ij} \ge 0$ — shipped quantity from facility $i$ to customer $j$

**Auxiliary (epigraph)**
- $\eta \in \mathbb{R}$ — upper bound on worst-case second-stage transportation cost

---

#### Two-Stage Robust Optimization (Min–Max–Min)
We consider a location–transportation system with uncertain customer demand captured by a budgeted polyhedral uncertainty set. First-stage decisions open facilities and install capacities; second-stage (recourse) decisions route flow to meet the realized demand.

$$
\begin{aligned}
\min_{(y,z)\in \mathcal{S}_y}\quad
& \sum_{i\in\mathcal{I}} f_i y_i \;+\; \sum_{i\in\mathcal{I}} a_i z_i
\;+\;
\underbrace{\max_{d \in \mathcal{D}}
\ \min_{x \in \mathcal{S}_x(d;y,z)}
\ \sum_{i\in\mathcal{I}}\sum_{j\in\mathcal{J}} c_{ij} x_{ij}}_{\text{worst-case recourse cost}}.
\end{aligned}
$$

#### Uncertainty Set (Budgeted Demand)

$$
\mathcal{D}
=
\left\{
d \in \mathbb{R}_+^{n} :
\ d_j \;=\; \bar d_j + g_j \tilde d_j,\;\; 0 \le g_j \le 1\ \forall j,\;\;
\sum_{j\in\mathcal{J}} g_j \le \Gamma
\right\}.
$$

#### First-Stage Feasible Set
$$
\mathcal{S}_y \;=\; \left\{(y,z) \in \{0,1\}^{m} \times \mathbb{R}_+^{m} \;:\; z_i \le K_i\, y_i \ \forall i \in \mathcal{I} \right\}.
$$

#### Second-Stage Feasible Set (for a realized $d \in \mathcal{D}$)
$$
\mathcal{S}_x(d;y,z) \;=\; \left\{
x \in \mathbb{R}_+^{m \times n} \;:\;
\sum_{j\in\mathcal{J}} x_{ij} \le z_i \ \forall i,\quad
\sum_{i\in\mathcal{I}} x_{ij} \ge d_j \ \forall j
\right\}.
$$

We assume **relatively complete recourse** (e.g., $\sum_{i} K_i \ge \max_{d\in \mathcal{D}} \sum_j d_j$) so that feasible second-stage solutions exist for any feasible first-stage decision. The following auxiliary model helps calculate the maximal total demand that could ever happen given the uncertainty set $\mathcal{D}$.

In [3]:
function get_maximal_demand(data)
    I = 1:data[:m] 
    J = 1:data[:n]
    d_nom, d_dev, Γ = data[:d_nom], data[:d_dev], data[:Γ]
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variables(model, begin
        d[j in J] >= 0
        0 <= g[j in J] <= 1
    end)

    @constraints(model, begin
        [j in J], d[j] == d_nom[j] + g[j] * d_dev[j]
        g[1] + g[2] + g[3] <= Γ[1]
        g[1] + g[2] <= Γ[2]
    end)
    
    @objective(model, Max, sum(d[j] for j in J))
    optimize!(model)
    return objective_value(model)
end

get_maximal_demand (generic function with 1 method)

#### Master Problem

To derive the master problem, introducing an epigraph variable $\eta$ for the worst-case second-stage cost:
$$
\begin{aligned}
\min_{(y,z)\in \mathcal{S}_y,\ \eta}\quad
& \sum_{i} f_i y_i + \sum_{i} a_i z_i + \eta \\
\text{s.t.}\quad
& \eta \;\ge\; \min_{x \in \mathcal{S}_x(d;y,z)} \sum_{i,j} c_{ij} x_{ij}
\quad \forall d \in \mathcal{D}.
\end{aligned}
$$

For more details about the master and subproblem formulations, please refer to the original paper and its supplementary materials. 

In [4]:
function build_master(d_scenarios, data)
    """
    build_master(g_scenarios, data) -> Model, y, z, η
    - d_scenarios: Vector of vectors, each of length n
        [matrix]: L by J 
    - data: Dict containing m, n, f, a, K, c
    Returns: JuMP.Model with variables y,z,η and recourse x[l]
    """
    I = 1:data[:m] 
    J = 1:data[:n]
    f, a, K, c = data[:f], data[:a], data[:K], data[:c]
    # d_nom, d_dev, Γ = data[:d_nom], data[:d_dev], data[:Γ]
    d = d_scenarios

    model = Model(Gurobi.Optimizer)
    set_silent(model)

    # First‐stage
    @variables(model, begin
        y[i in I], Bin
        z[i in I] >= 0
        η
    end)

    # Recourse vars for each scenario l
    L = 1:length(d_scenarios)
    @variables(model, begin
        x[l in L, i in I, j in J] >= 0
    end)

    @constraints(model, begin
        [i in I], z[i] <= K * y[i]
        [l in L], η >= sum(c[i,j]*x[l,i,j] for i in I, j in J)
        [l in L, j in J], sum(x[l,i,j] for i in I) >= d[l][j]
        [l in L, i in I], sum(x[l,i,j] for j in J) <= z[i]
    end)
    
    # Total capacity is greater than the maximal total demand. 
    z_min = get_maximal_demand(data)
    @constraint(model, sum(z[i] for i in I) >= z_min)

    # Objective (MP)
    @objective(model, Min,
        sum(f[i]*y[i] for i in I)
      + sum(a[i]*z[i] for i in I)
      + η
    )

    optimize!(model)
    return model, y, z, η
end


build_master (generic function with 1 method)

#### Subproblem

In [5]:
function build_subproblem_milp(z_val, data)
    """
    build_subproblem(z_val, data) -> Model, g, xsp
    - z_val: vector of length m, the fixed capacity decisions
    - data: Dict containing m, n, c, d_nom, d_dev, Γ
        m: indices i=1, 2, 3
        n: indices j=1, 2, 3
        c: [matrix]
        d_nom: [R^3]
        d_dev: [R^3]
        Γ: [R^2] RHS of two constraints in D

    Returns: JuMP.Model with variables g[1:n], xsp[1:m,1:n]
    """
    I = 1:data[:m] 
    J = 1:data[:n]
    c, d_nom, d_dev, Γ = data[:c], data[:d_nom], data[:d_dev], data[:Γ]

    model = Model(Gurobi.Optimizer)
    set_silent(model)

    @variables(model, begin
        d[j in J] >= 0
        π[i in I] >= 0
        λ[j in J] >= 0
        x[i in I, j in J] >= 0
        0 <= g[j in J] <= 1
        α[i in I, j in J], Bin
        β[j in J], Bin
        γ[i in I], Bin
    end)

    # Big-M relaxation
    M = 1000
    @constraints(model, begin
        [i in I], sum(x[i,j] for j in J) <= z_val[i]
        [j in J], sum(x[i,j] for i in I) <= d[j]
        [i in I, j in J], λ[j] - π[i] <= c[i,j]
        [i in I, j in J], x[i,j] <= M * α[i,j]
        [i in I, j in J], (c[i,j] - λ[j] + π[i]) <= (c[i,j]*M)*(1-α[i,j])
        [j in J], λ[j] <= M * β[j]
        [j in J], (sum(x[i,j] for i in I) - d[j]) <= d_dev[j] * (1-β[j])
        [i in I], π[i] <= M * γ[i]
        [i in I], (z_val[i] - sum(x[i,j] for j in J)) <= z_val[i] * (1-γ[i])
        [j in J], d[j] == d_nom[j] + g[j] * d_dev[j]
        g[1] + g[2] + g[3] <= Γ[1]
        g[1] + g[2] <= Γ[2]
    end)

    @objective(model, Max,
        sum(c[i,j] * x[i,j] for i in I, j in J)
    )

    optimize!(model)

    return model, d, λ, π, x
end

build_subproblem_milp (generic function with 1 method)

#### Implementation of the C\&CG Algorithm

In [6]:
function col_constr_gen()
    mp_list = JuMP.Model[]      # for masters
    sp_list = JuMP.Model[]      # for subproblems

    LB, UB, k = -Inf, Inf, 0
    convergence = false
    while !convergence
        # 1. build & solve master
        mp, y, z, η = build_master(d_scenarios, data)
        push!(mp_list, mp)       # save it

        LB = objective_value(mp)
        println("Lower bound: $LB")
        y_val, z_val = value.(y), value.(z)

        # 2. build & solve subproblem
        sp, d, λ, π, x = build_subproblem_milp(z_val, data)
        push!(sp_list, sp) 

        Q = objective_value(sp)
        UB = min(UB, sum(f[i]*y_val[i] + a[i]*z_val[i] for i in 1:m) + Q)
        println("Upper bound: $UB")
        
        # 3. record new scenario
        push!(d_scenarios, value.(d))
        k += 1
        if (UB - LB) <= 1e-4
            convergence = true
            return mp, sp
        end
    end
end

col_constr_gen (generic function with 1 method)

#### Solution and the visualization of uncertainty set

$$
\mathcal{D} =
\Big\{
d :
d_0 = 206 + 40 g_0,\;
d_1 = 274 + 40 g_1,\;
d_2 = 220 + 40 g_2, \\ 
0 \le g_0 \le 1,\;
0 \le g_1 \le 1,\;
0 \le g_2 \le 1, \\
g_0 + g_1 + g_2 \le 1.8,\;
g_0 + g_1 \le 1.2
\Big\}.
$$

Since we only have three uncertain demands, the uncertainty set as a polyhedron can be visualized in 3-D space leveraging Polyhedra.jl. Notice that we visualize $d_i$ given the relations of $g_i$, so the halfspaces need some derivations. For example, the first halfspace $d_0+d_1+d_2 \le 772$ comes from $g_0+g_1+g_2 \le 1.8$:

\begin{aligned}
    d_0+d_1+d_2 & = (206 + 40 g_0) + (274 + 40 g_1) + (220 + 40 g_2) \\
    & = 700 + 40(g_0+g_1+g_2) \\
    & \le 700 + 40*1.8 \\
    & = 772
\end{aligned}

In [ ]:
mp, sp = col_constr_gen()

planes = [
    HalfSpace([ 1.0,  1.0,  1.0], 772.0),   # g0 + g1 + g2 ≤ 1.8
    HalfSpace([ 1.0,  1.0,  0.0], 528.0),   # g0 + g1 ≤ 1.2
    HalfSpace([ 1.0,  0.0,  0.0], 246.0),   # g0 ≤ 1
    HalfSpace([-1.0,  0.0,  0.0], -206.0),  # -g0 ≤ 0
    HalfSpace([ 0.0,  1.0,  0.0], 314.0),   # g1 ≤ 1
    HalfSpace([ 0.0, -1.0,  0.0], -274.0),  # -g1 ≤ 0
    HalfSpace([ 0.0,  0.0,  1.0], 260.0),   # g2 ≤ 1
    HalfSpace([ 0.0,  0.0, -1.0], -220.0),  # -g2 ≤ 0
]

hrepr = reduce(∩, planes)
p = polyhedron(reduce(∩, planes), CDDLib.Library())
m = Polyhedra.Mesh(p)

fig = Figure(resolution = (600, 600))
ax  = Axis3(fig[1, 1]; xlabel="ξ1", ylabel="ξ2", zlabel="ξ3", perspectiveness = 0.75)

# Filled polyhedron
mesh!(ax, m; transparency = true)
wireframe!(ax, m; linewidth = 1)

# Now plot the two points in red:
xs = [206.0, 206.0]
ys = [274.0, 314.0]
zs = [220.0, 252.0]

scatter!(ax, xs, ys, zs;
    markersize = 15,
    color      = :red,
    label      = "sample points"
)

for (x,y,z) in zip(xs, ys, zs)
    text!(ax, string("(",x,",",y,",",z,")"), position = (x,y,z),
          align = (:left, :bottom), fontsize = 12)
end
# save("./plots/transportation_uncer_3d.png", fig, px_per_unit = 8.0)

fig

Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Lower bound: 31832.0
Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Upper bound: 33680.0
Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Lower bound: 33680.0
Set parameter Username
Set parameter LicenseID to value 2679099
Academic license - for non-commercial use only - expires 2026-06-18
Upper bound: 33680.0


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie /Users/zheli/.julia/packages/Makie/aJUtI/src/scenes.jl:259


**Take-away:**
1. The C\&CG algorithm is extremely efficient for two-stage RO problems. In this case study, it converges in only 2 iterations. 
2. C\&CG expolits a key property of polyhedral uncertainty sets: the worst-case always lies at one of the extreme points. The algorithm iteratively solves MP to obtains a candidate first-stage decision, throws it into SP to identify a worst-case extreme point, and adds this realization back to MP. This procedure continues until the optimality gap reaches zero, i.e., the worst realization is found. 
> One beautiful thing about solving linear problems is that we can turn a infinite continous search into a finite searching over extreme points, which we have already found in simplex algorithm. 